In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/xindai/test_split.csv
/kaggle/input/xindai/test_unlabel.csv
/kaggle/input/xindai/xindai/xindai/dev.csv
/kaggle/input/xindai/xindai/xindai/train.csv
/kaggle/input/xindai/xindai/xindai/test.csv
/kaggle/input/stsbenchmark/COVID_Ethics_dev.csv
/kaggle/input/stsbenchmark/COVID_Ethics_train.csv
/kaggle/input/stsbenchmark/COVID_Ethics_raw.csv
/kaggle/input/stsbenchmark/COVID_Ethics_all.csv
/kaggle/input/stsbenchmark/stsbenchmark/stsbenchmark/sts-train-dev.csv
/kaggle/input/stsbenchmark/stsbenchmark/stsbenchmark/sts-train.csv
/kaggle/input/stsbenchmark/stsbenchmark/stsbenchmark/readme.txt
/kaggle/input/stsbenchmark/stsbenchmark/stsbenchmark/LICENSE.txt
/kaggle/input/stsbenchmark/stsbenchmark/stsbenchmark/sts-dev.csv
/kaggle/input/stsbenchmark/stsbenchmark/stsbenchmark/sts-test.csv
/kaggle/input/stsbenchmark/stsbenchmark/stsbenchmark/correlation.pl
/kaggle/input/dialog-match-trian-data/dialog_match_train_data/train_split.csv
/kaggle/input/dialog-match-trian-data/dialog_match_train

In [2]:
!pip install sentence_transformer

ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer


In [3]:
import sys 
sys.path.append('/kaggle/input/sentencebert/sentence-transformers-master')
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util

In [4]:
from torch.utils.data import DataLoader
import math
import sys
sys.path.append('/kaggle/input/sentencebert/sentence-transformers-master')
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import pandas as pd
print('---import finished')

---import finished


In [5]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sklearn.metrics import accuracy_score

## 使用没有fine-tune的模型直接进行相似度判断，准确度 0.81

In [6]:
def sbert_acc():
    #df_data = pd.read_csv('../input/xindai/test_unlabel.csv', names=['sent1','sent2'],sep='\t')
    df_data = pd.read_csv('../input/xindai/test_split.csv', names=['sent1','sent2','label'],sep='\t')
    #df_data = df_data.head(10)
    text = df_data[['sent1','sent2']].values.tolist()
    #print('--text',text)
    from transformers import BertTokenizer, BertModel
    from sklearn.metrics.pairwise import cosine_similarity
    import torch

    model = BertModel.from_pretrained("uer/sbert-base-chinese-nli")
    tokenizer = BertTokenizer.from_pretrained("uer/sbert-base-chinese-nli")
    all_result =[]
    for i in range(len(text)):
        sentences = text[i]
        #print('----sentences',sentences)
        # 初始化字典来存储
        #sentences = ['什么的涟漪', '什么的的涟漪']
        tokens = {'input_ids': [], 'attention_mask': []}
        for sentence in sentences:
            new_tokens = tokenizer.encode_plus(sentence, max_length=15, truncation=True, padding='max_length',
                                               return_tensors='pt')
            tokens['input_ids'].append(new_tokens['input_ids'][0])
            tokens['attention_mask'].append(new_tokens['attention_mask'][0])

        # 将张量列表重新格式化为一个张量
        tokens['input_ids'] = torch.stack(tokens['input_ids'])
        tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

        outputs = model(**tokens)
        # print(outputs.keys()) #odict_keys(['last_hidden_state', 'pooler_output'])

        embeddings = outputs.last_hidden_state
        # print(embeddings.shape) #torch.Size([4, 15, 768])

        attention_mask = tokens['attention_mask']
        # print(attention_mask.shape) #torch.Size([4, 15])

        mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
        # print(mask.shape) #torch.Size([4, 15, 768])

        masked_embeddings = embeddings * mask
        # print(masked_embeddings.shape) #torch.Size([4, 15, 768])

        summed = torch.sum(masked_embeddings, 1)
        # print(summed.shape) #torch.Size([4, 768])

        summed_mask = torch.clamp(mask.sum(1), min=1e-9)
        # print(summed_mask.shape) #torch.Size([4, 768])

        mean_pooled = summed / summed_mask
        # print(mean_pooled.shape) #torch.Size([4, 768])

        mean_pooled = mean_pooled.detach().numpy()
        result = cosine_similarity([mean_pooled[0]], [mean_pooled[1]])[0][0]
        #print(result)  # [[0.9864919 0.39011386 0.29779416]]
        all_result.append(result)
    df_data['cos']=all_result
    print(df_data.head(10))
    df_data.to_csv('./submit_result.txt', index=False, sep='\t')
    return df_data

In [7]:
# import numpy as np
# df_df_result = pd.read_csv('./submit_result.txt',  sep='\t')
# #df_result = sbert_acc() # 没有fine-tune的sbert
# from sklearn.metrics import accuracy_score
# df_result['sim_flag'] = df_result['cos'].apply(lambda x: 1 if x >= 0.75 else 0)
# print(df_result.head(10))

# y_pre= np.array(df_result['sim_flag'].values.tolist())
# y_true = np.array(df_result['label'].values.tolist())

# print(y_pre[:10])
# print(y_true[:10])
# print(accuracy_score(y_true, y_pre))

In [8]:


"""
This examples trains a CrossEncoder for the STSbenchmark task. A CrossEncoder takes a sentence pair
as input and outputs a label. Here, it output a continious labels 0...1 to indicate the similarity between the input pair.

It does NOT produce a sentence embedding and does NOT work for individual sentences.

Usage:
python training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv

## 用比赛数据进行训练

In [9]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout


# Check if dataset exsist. If not, download and extract  it
# sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

# if not os.path.exists(sts_dataset_path):
#     util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)


# Define our Cross-Encoder
train_batch_size = 16
num_epochs = 1

model_save_path = 'output/training_stsbenchmark-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# We use distilroberta-base as base model and set num_labels=1, which predicts a continous score between 0 and 1
# model = CrossEncoder('uer/sbert-base-chinese-nli', num_labels=1)
model = CrossEncoder('distilroberta-base', num_labels=1)

# Read STSb dataset
logger.info("Read STSbenchmark train dataset")

train_samples = []
dev_samples = []
test_samples = []
df_train = pd.read_csv('../input/dialog-match-trian-data/dialog_match_train_data/data_for_pretrain.txt', sep='\t')
df_train.dropna(inplace=True)
#df_train = df_train.head(1000)
print(df_train.shape, df_train.head(3))
data = df_train.values.tolist()
for index in range(len(data)):
    try:
        train_samples.append(InputExample(texts=[data[index][0], data[index][1]], label=data[index][2]))
        train_samples.append(InputExample(texts=[data[index][1], data[index][0]], label=data[index][2]))
    except:
        print(data[index])
print('-----train', len(train_samples))

df_dev = pd.read_csv('../input/dialog-match-trian-data/dialog_match_train_data/dev_split.csv', sep='\t')
print(df_dev.shape, df_dev.head(3))
data = df_dev.values.tolist()
for index in range(len(data)):
    try:
        dev_samples.append(InputExample(texts=[data[index][0], data[index][1]], label=data[index][2]))
    except:
        print(data[index])
print('-----dev', len(dev_samples))

df_test = pd.read_csv('../input/dialog-match-trian-data/dialog_match_train_data/test_split.csv', sep='\t')
print(df_test.shape, df_test.head(3))
data = df_test.values.tolist()
for index in range(len(data)):
    try:
        test_samples.append(InputExample(texts=[data[index][0], data[index][1]], label=data[index][2]))
    except:
        print(data[index])
print('-----test', len(test_samples))

# We wrap train_samples (which is a List[InputExample]) into a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# We add an evaluator, which evaluates the performance during training
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')

# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

##### Load model and eval on test set
model = CrossEncoder(model_save_path)

evaluator = CECorrelationEvaluator.from_input_examples(test_samples, name='sts-test')
evaluator(model)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

(632023, 3)              sent1            sent2  label
0      怎么更换手机充电提示音  怎么更换手机充满电后的提示音。      1
1  怎样向一个自己喜欢的男生表白？    喜欢一个男生，该如何表白？      1
2    OPPOa52怎么看分辨率          a52常见问题      0
-----train 1264046
(2500, 3)              sent1                        sent2  label
0  好的，您在5分钟内回复一下我哦  您好，麻烦您5分钟左右到邮箱中查收一下邮件操作修改即可      0
1        推荐好看的清穿小说                     请推荐好看的小说      1
2      姓杨的男网络红人，帅哥                   有哪些网络红人是帅哥      0
-----dev 2500
(2500, 3)          sent1           sent2  label
0        什么的涟漪          什么的的涟漪      1
1  硬头发男生适合什么发型  男生头发又干又柴适合什么发型      0
2      这是什么木头。       这是什么木头做的啊      0
-----test 2500


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/79003 [00:00<?, ?it/s]

0.7988879048436612

In [10]:
# from sklearn.metrics import accuracy_score
# df_result['sim_flag'] = df_result['cos'].apply(lambda x: 1 if x >= 0.75 else 0)
# y_pre = np.array(df_result['sim_flag'].values.tolist())
# y_true = np.array(df_result['label'].values.tolist())
# print(accuracy_score(y_true, y_pre))

In [11]:
# model_name = 'hfl/chinese-roberta-wwm-ext-large'
# train_batch_size = 16
# num_epochs = 4
# model_save_path = 'test_output'
# logging.basicConfig(format='%(asctime)s - %(message)s',
#   datefmt='%Y-%m-%d %H:%M:%S',
#   level=logging.INFO,
#   handlers=[LoggingHandler()])

# # Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
# word_embedding_model = models.Transformer(model_name)

# # Apply mean pooling to get one fixed sized sentence vector
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
#   pooling_mode_mean_tokens=True,
#   pooling_mode_cls_token=False,
#   pooling_mode_max_tokens=False)

# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# train_samples = []
# dev_samples = []
# test_samples = []

# def load(path):
#   df = pd.read_csv(path)
#   samples = []
#   for idx,item in df.iterrows():
#     samples.append(InputExample(texts=[item['sentence1'], item['sentence2']], label=float(item['label'])))
#     samples = samples[:1000]
#   return samples

# data_dir = '/data/keraszhou/ask_match/data/base_data/xindai/'
# train_samples = load('/kaggle/input/xindai/xindai/train.csv')
# test_samples = load('/kaggle/input/xindai/xindai/dev.csv')
# dev_samples = load( '/kaggle/input/xindai/xindai/test.csv')

# train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
# train_loss = losses.CosineSimilarityLoss(model=model)
# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')
# warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

# # Train the model
# model.fit(train_objectives=[(train_dataloader, train_loss)],
#   evaluator=evaluator,
#   epochs=num_epochs,
#   evaluation_steps=1000,
#   warmup_steps=warmup_steps,
#   output_path=model_save_path)

# model = SentenceTransformer(model_save_path)
# test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
# test_evaluator(model, output_path=model_save_path)